<a href="https://colab.research.google.com/github/DavidDaimond/ds_school_2020/blob/master/task%205/Homework_5_Strokatov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import imdb
from keras import preprocessing
import tensorflow_datasets as tfds

Using TensorFlow backend.


In [0]:
#Yea, we import 2 datasets - one for sklearn and one for tensorflow/keras models

ds, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_ex, test_ex = ds['train'], ds['test']

max_features = 10000
maxlen = 25

In [0]:
train_ex = [x for x in train_ex.as_numpy_iterator()]
test_ex = [x for x in test_ex.as_numpy_iterator()] #Transform data to a numpy arrays

In [0]:
train_ex[:5]

[(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
  0),
 (b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot deve

In [0]:
#divide it to the
x_train, y_train = np.array([x[0] for x in train_ex]), np.array([y[1] for y in train_ex])
x_test, y_test = np.array([x[0] for x in test_ex]), np.array([y[1] for y in test_ex])

#look at the random 
x_train[42], y_train[42], x_test[42], y_test[42]

(b'One of the weaker Carry On adventures sees Sid James as the head of a crime gang stealing contraceptive pills. The fourth of the series to be hospital-based, it\'s possibly the least of the genre. There\'s a curiously flat feel throughout, with all seemingly squandered on below-par material. This is far from the late-70s nadir, but Williams, James, Bresslaw, Maynard et al. are all class performers yet not given the backing of a script equal to their ability.<br /><br />Most of the gags are onrunning, rather than episodic as Carry Ons usually are. So that instead of the traditional hit and miss ratio, if you don\'t find the joke funny in the first place you\'re stuck with it for most of the film. These continuous plot strands include Williams \xc2\x96 for no good reason \xc2\x96 worrying that he\'s changing sex, and Kenneth Cope in drag. Like the stagy physical pratt falls, the whole thing feels more contrived than in other movies, and lacking in cast interest. Continuing this theme,

In [0]:
y_train.dtype

dtype('int64')

In [0]:
y_test

array([1, 1, 0, ..., 0, 1, 1])

## **Data analysis**

In [0]:
#As we can see - train dataset is half-positive (or half-negative)
print(len(y_train),
sum(y_train),
len(y_train)-sum(y_train))

25000 12500 12500


In [0]:
#And also for test dataset! Extraordinary!
print(len(y_test),
sum(y_test),
len(y_test)-sum(y_test))

25000 12500 12500


In [0]:
#let's look to our reviews
print(x_train.dtype,
      x_train[:5])

|S13704 [b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot devel

# **Tfidf and logreg tests**

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

In [0]:
%%time
tfidf = TfidfVectorizer()
tfidf.fit_transform(x_train)

tx_train = tfidf.transform(x_train)
tx_test = tfidf.transform(x_test)
tx_test[42]

CPU times: user 13.6 s, sys: 92.9 ms, total: 13.6 s
Wall time: 13.7 s


In [0]:
logreg = LogisticRegression(random_state=42)

logreg.fit(tx_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
#mean accuracy of logistic regression

logreg.score(tx_test, y_test)

0.88316

# **Tokenizer and dense tests**

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

In [0]:
tokenizer = Tokenizer(10000)
tokenizer.fit_on_texts([str(x) for x in x_train])

In [0]:
tokenizer.get_config()

In [0]:
tokx_train = tokenizer.texts_to_matrix([str(x) for x in x_train])
tokx_test = tokenizer.texts_to_matrix([str(x) for x in x_test])

tokx_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [0]:
tokx_train.shape

(25000, 10000)

In [0]:
#build a network, as output activation function we put sigmoid
model1 = Sequential()

model1.add(Dense(2048, input_shape=(10000,), activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(1, activation='sigmoid'))

In [0]:
model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 2048)              20482048  
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 21,006,849
Trainable params: 21,006,849
Non-trainable params: 0
_________________________________________________________________


In [0]:
model1.compile(loss=BinaryCrossentropy(),
             optimizer=Adam(1e-3),
             metrics=['accuracy'])

In [0]:
%%time
history1 = model1.fit(tokx_train, y_train, epochs=4, batch_size=1000)

Epoch 1/4
25/25 [==============================] - 35s 1s/step - loss: 0.4388 - accuracy: 0.7836
Epoch 2/4
25/25 [==============================] - 36s 1s/step - loss: 0.1917 - accuracy: 0.9296
Epoch 3/4
25/25 [==============================] - 34s 1s/step - loss: 0.1017 - accuracy: 0.9658
Epoch 4/4
25/25 [==============================] - 34s 1s/step - loss: 0.0398 - accuracy: 0.9896
CPU times: user 4min 41s, sys: 1.02 s, total: 4min 42s
Wall time: 2min 26s


In [0]:
predict1 = model1.predict(tokx_test)

In [0]:
#Now count the accuracy...
(predict1.round().ravel() == y_test).mean()

0.87384

In [0]:
#Making a dense network have a little profit, because we have accuracy less than logreg....
#RNN, I choose you!

# **Embedding and RNN learning**

In [0]:
from tensorflow.keras.layers import Embedding, SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
#but first time we need to rework data for RNN by padding
seqx_train = pad_sequences(tokenizer.texts_to_sequences([str(x) for x in x_train]))
seqx_test = pad_sequences(tokenizer.texts_to_sequences([str(x) for x in x_test]))

In [0]:
model2 = Sequential()

model2.add(Embedding(10000, 5))
model2.add(SimpleRNN(10))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 5)           50000     
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 10)                160       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 11        
Total params: 50,171
Trainable params: 50,171
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.compile(loss=BinaryCrossentropy(),
               optimizer=Adam(1e-3),
               metrics=['accuracy'])

In [0]:
%%time
history2 = model2.fit(seqx_train, y_train, epochs=10, batch_size=100)

Epoch 1/10
250/250 [==============================] - 209s 836ms/step - loss: 0.6187 - accuracy: 0.6724
Epoch 2/10
250/250 [==============================] - 213s 852ms/step - loss: 0.4601 - accuracy: 0.8003
Epoch 3/10
250/250 [==============================] - 216s 865ms/step - loss: 0.3799 - accuracy: 0.8464
Epoch 4/10
250/250 [==============================] - 214s 857ms/step - loss: 0.3299 - accuracy: 0.8713
Epoch 5/10
250/250 [==============================] - 213s 852ms/step - loss: 0.2757 - accuracy: 0.8968
Epoch 6/10
250/250 [==============================] - 211s 844ms/step - loss: 0.2319 - accuracy: 0.9188
Epoch 7/10
250/250 [==============================] - 213s 853ms/step - loss: 0.1970 - accuracy: 0.9330
Epoch 8/10
250/250 [==============================] - 214s 857ms/step - loss: 0.1771 - accuracy: 0.9422
Epoch 9/10
250/250 [==============================] - 212s 846ms/step - loss: 0.1576 - accuracy: 0.9502
Epoch 10/10
250/250 [==============================] - 215s 859m

## Tests

# **Graphics, metrics and comparing**